SPAM FILTERING USING MACHINE LEARNING CLASSIFICATION

In [27]:
from os import listdir
from os.path import isfile, join

from sklearn import datasets
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

import nltk
from nltk.tokenize import word_tokenize          #DOESN'T REMOVE NUMBERS OR SPECIAL CHARACTERS
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import numpy as np
from collections import Counter

from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.metrics import confusion_matrix

In [28]:
#SAMPLE FOR IRIS DATASET

iris = datasets.load_iris()

bnb = BernoulliNB()
y_pred = bnb.fit(iris.data, iris.target).predict(iris.data)
print("For BernoulliNB, Number of mislabeled points out of a total %d points : %d" % (iris.data.shape[0],(iris.target != y_pred).sum()))

gnb = GaussianNB()
y_pred = gnb.fit(iris.data, iris.target).predict(iris.data)
print("For GaussianNB, Number of mislabeled points out of a total %d points : %d" % (iris.data.shape[0],(iris.target != y_pred).sum()))

mnb = MultinomialNB()
y_pred = mnb.fit(iris.data, iris.target).predict(iris.data)
print("For MultinomialNB, Number of mislabeled points out of a total %d points : %d" % (iris.data.shape[0],(iris.target != y_pred).sum()))

For BernoulliNB, Number of mislabeled points out of a total 150 points : 100
For GaussianNB, Number of mislabeled points out of a total 150 points : 6
For MultinomialNB, Number of mislabeled points out of a total 150 points : 7


In [29]:
mypath = r"C:\Users\iis.B104-47\Desktop\enron1\spam"
spamfiles = [(join(mypath, f)) for f in listdir(mypath) if isfile(join(mypath, f))]

print(len(spamfiles))

1500


In [30]:
mypath = r"C:\Users\iis.B104-47\Desktop\enron1\ham"
hamfiles = [(join(mypath, f)) for f in listdir(mypath) if isfile(join(mypath, f))]

print(len(hamfiles))

3672


In [33]:
#CLEAN ALL SENTENCES BY EXTRACTING TOKENS FROM EACH WORD AND USING TOP 3000 OF THEM AS FEATURES

stopset = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')
datasett = []
all_words = []

for f in spamfiles:
    with open (f, "r") as myfile:
        #print(f)
        x = myfile.read().replace('\n', '').lower()
        #tokens = word_tokenize(x)
        tokens = tokenizer.tokenize(x)

    tokens = [w for w in tokens if not w in stopset]
    datasett.append([f, 'spam'])
    for w in tokens:
        all_words.append(w)
    
for f in hamfiles:
    with open (f, "r") as myfile:
        x = myfile.read().replace('\n', '').lower()
        #tokens = word_tokenize(x)
        tokens = tokenizer.tokenize(x)

    tokens = [w for w in tokens if not w in stopset]
    datasett.append([f, 'ham'])
    for w in tokens:
        all_words.append(w)
    
dictionary = Counter(all_words)
list_to_remove = dictionary.keys()

for d in list_to_remove:
    if len(d) == 1:
        dictionary[d] = 0

dictionary = dictionary.most_common(3000)
print(dictionary)

#print(datasett)
print(len(datasett))

[('ect', 11647), ('hou', 7283), ('subject', 6668), ('enron', 5588), ('2000', 3303), ('com', 2976), ('gas', 2771), ('please', 2645), ('deal', 2528), ('meter', 2234), ('hpl', 2160), ('000', 2005), ('10', 1801), ('2001', 1766), ('corp', 1754), ('00', 1704), ('thanks', 1563), ('know', 1468), ('01', 1411), ('ectcc', 1386), ('need', 1370), ('pmto', 1339), ('new', 1320), ('forwarded', 1292), ('may', 1286), ('daren', 1236), ('11', 1221), ('mmbtu', 1199), ('get', 1150), ('12', 1128), ('see', 1119), ('amto', 1094), ('http', 1090), ('let', 1074), ('price', 1041), ('xls', 1014), ('company', 1010), ('03', 971), ('would', 969), ('attached', 966), ('information', 926), ('farmer', 914), ('99', 911), ('us', 905), ('message', 896), ('day', 890), ('time', 883), ('one', 866), ('30', 851), ('mail', 823), ('th', 816), ('02', 811), ('volume', 803), ('20', 803), ('04', 784), ('contract', 780), ('robert', 779), ('sitara', 770), ('month', 763), ('texas', 749), ('nom', 741), ('05', 741), ('pm', 731), ('ectsubjec

In [34]:
#CREATE FEATURE MATRICES FOR BOTH TRAINING AND TESTING SETS
train_feat_matrix = np.zeros((3879,3000))
test_feat_matrix= np.zeros((1293,3000))
docID = 0;
train_files = spamfiles[0:1125]+hamfiles[0:2754]
test_files = spamfiles[1125:]+hamfiles[2754:]

train_labels = np.zeros(3879)
train_labels[1125:3879] = 1

test_labels = np.zeros(1293)
test_labels[375:1293] = 1

print(len(test_files), train_labels[1126])

for fil in train_files:
    with open(fil, "r") as fi:
        x = fi.read().replace('\n', '').lower()
        #tokens = word_tokenize(x)
        words = tokenizer.tokenize(x)
        for word in words:
            wordID = 0
            for i,d in enumerate(dictionary):
                if d[0] == word:
                    wordID = i
                    train_feat_matrix[docID,wordID] = words.count(word)
    docID = docID + 1     
print(train_feat_matrix)

docID = 0
for fil in test_files:
    with open(fil, "r") as fi:
        x = fi.read().replace('\n', '').lower()
        #tokens = word_tokenize(x)
        words = tokenizer.tokenize(x)
        for word in words:
            wordID = 0
            for i,d in enumerate(dictionary):
                if d[0] == word:
                    wordID = i
                    test_feat_matrix[docID,wordID] = words.count(word)
    docID = docID + 1 
print(test_feat_matrix)

(1293, 1.0)
[[ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]]
[[ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  2. ...,  0.  0.  0.]]


In [43]:
models = [MultinomialNB(), GaussianNB(), BernoulliNB(), NuSVC()]
names = ['MultinomialNB', 'GaussianNB', 'BernoulliNB', 'NuSVC']

for i,model in enumerate(models):
               
    model.fit(train_feat_matrix,train_labels)
    result = model.predict(test_feat_matrix)
    conf1 = confusion_matrix(test_labels,result)
    print '\nFor'+names[i]+'\n',conf1
    print 'Accuracy is: '+ str(float(conf1[0][0] + conf1[1][1])/12.93)


ForMultinomialNB
[[362  13]
 [105 813]]
Accuracy is: 90.8739365816

ForGaussianNB
[[352  23]
 [ 78 840]]
Accuracy is: 92.18870843

ForBernoulliNB
[[375   0]
 [124 794]]
Accuracy is: 90.4098994586

ForNuSVC
[[305  70]
 [ 31 887]]
Accuracy is: 92.18870843


In [36]:
print(test_feat_matrix.shape, train_feat_matrix.shape)

((1293, 3000), (3879, 3000))
